# SPLAT Tutorials: Basic Spectral Analysis

## Authors
Adam Burgasser

## Learning Goals
* Read in a spectrum from the SPLAT database or externally (splat.searchLibrary, splat.getSpectrum)
* Plot a spectrum (splat.Spectrum.plot)
* Some basic manipulation of spectra - normalizing, scaling, trimming, changing units, spectral math (splat.Spectrum)
* Flux calibrate a spectrum (splat.Spectrum.fluxCalibrate)
* Measure spectrophotometry from a spectrum (splat.Spectrum.filterMage, splat.photometry.filterMag)
* Measure a single and set of indices (splat.measureIndex, splat.measureIndexSet)
* Use indices to infer a spectrum's spectral type (splat.classifyByIndex)
* Compare a spectrum to another spectrum (splat.compareSpectrum)
* Compare a spectrum a set of spectral standards (classifyByStandard)

## Keywords
spectral archive, spectral analysis, indices, classification

## Companion Content
None

## Summary
In this tutorial, we will examine how to draw a spectrum from the SPLAT library and conduct some basic spectral analyses to that object, including visualization, manipulation of the spectrum, using photometry to flux calibrate or measure the colors of a spectrum, measure spectral indices, and classification.


# Starting off

Let's make sure the code is properly downloaded through the import statements; see http://pono.ucsd.edu/~adam/browndwarfs/splat/ for more detail on the proper installation procedures

In [ ]:
# main splat import
import splat
import splat.plot as splot
import splat.photometry as sphot
import splat.empirical as spem

# other useful imports
import matplotlib.pyplot as plt
import numpy as np
import pandas
import astropy.units as u
from astropy.io import fits
from astropy.utils.data import download_file


In [ ]:
# check what version you are using
splat.VERSION

In [ ]:
# check that you have some spectra in the library
print(len(splat.DB_SPECTRA))

In [ ]:
# who has contributed to this code?
splat.AUTHORS

# Reading in and visualizing spectra

SPLAT contains a built-in library of published SpeX prism spectra of ultracool dwarfs. It is also possible to download additional spectral datasets and read in your own spectrum or a spectrum from an website. Once you've read a spectrum into a Spectrum object, you can use the built-in features to visualize the spectrum.

In [ ]:
# grab a random spectrum from the library and plot it
# this produces a list of Spectrum objects so we want just the first one
sp = splat.getSpectrum(lucky=True)[0]
sp.plot()

In [ ]:
# get some information about this spectrum using info()
sp.info()

In [ ]:
# grab a random L5 dwarf
# this produces a list of Spectrum objects so we want just the first one
sp = splat.getSpectrum(spt='L5', lucky=True)[0]
sp.plot()

In [ ]:
# grab a very specific spectrum based on its source ID
sp = splat.Spectrum(10001)
sp.plot()

In [ ]:
# grab all the spectra of TWA 30A
splist = splat.getSpectrum(name='TWA 30A')
print(splist)
for sp in splist: sp.plot()

In [ ]:
# we can also search the library for spectra
# this produces a pandas table of the relevant spectra
s = splat.searchLibrary(name='TWA30B')
s

In [ ]:
# choose one of these spectra
sp = splat.Spectrum(s['DATA_KEY'][1])
sp.plot()

In [ ]:
# read in a spectrum from an online fits file
f = download_file('http://pono.ucsd.edu/~adam/data/spex_test/spex_prism_PSOJ0077921+578267_120924.fits')
sp = splat.Spectrum(file=f,file_type='fits',name='PSOJ0077921+578267')
sp.plot()

# Spectrum manipulation

There are many built-in features for manipulating a spectrum object

In [ ]:
# grab a random T5 dwarf
sp = splat.getSpectrum(spt='T5', lucky=True)[0]
sp.plot()

In [ ]:
# normalize the spectrum to maximum value
sp.normalize()
sp.plot()

In [ ]:
# normalize over a specific region
sp.normalize([1.5,1.7])
sp.plot()

In [ ]:
# multiple by a scale factor
sp.scale(50)
sp.plot()

In [ ]:
# flux calibrate the spectrum using a photometric magnitude
# form SpeX prism spectra these should be filters in the 1-2.5 micron range
# such as 2MASS JHKs, UKIDSS JHK, HST F110W/F160W, etc.
sp.fluxCalibrate('2MASS J',14.5,absolute=True)  # the "absolute" flag indicates this is an absolute magnitude
sp.plot()

In [ ]:
# trim the edges
sp.trim([0.9,2.3])
sp.plot()

In [ ]:
# mask part of the spectrum
sp.maskFlux([1.8,2.0])
sp.plot()

In [ ]:
# change the wavelength units
sp.toWaveUnit(u.Angstrom)
sp.plot()

In [ ]:
# change the flux units
sp.toFluxUnit(u.W/u.m/u.m/u.Angstrom)
sp.plot()

In [ ]:
# reset all your changes to go back to the original spectrum
sp.reset()
sp.plot()

# Spectral Math

The Spectrum object takes care of all of the necessary math steps to add, subtract, multiply and divide spectra

In [ ]:
# read in two M-type spectra, normalize them and add them together
sp1 = splat.getSpectrum(spt=['M5','M9'],lucky=True)[0]
sp2 = splat.getSpectrum(spt=['M5','M9'],lucky=True)[0]
sp1.normalize()
sp2.normalize()

# add together
sp3 = sp1+sp2

# plot this up using matplotlib
plt.plot(sp1.wave,sp1.flux,'b-')
plt.plot(sp2.wave,sp2.flux,'g-')
plt.plot(sp3.wave,sp3.flux,'k-')
plt.legend([sp1.name,sp2.name,'Sum'])
plt.ylim([0,2.2])
plt.xlim([0.8,2.4])
plt.xlabel('Wavelength (micron)')
plt.ylabel('Normalized Flux Density')


In [ ]:
# read in two M7 spectra, normalize them and subtract them to see differences
sp1 = splat.getSpectrum(spt='M7',lucky=True)[0]
sp2 = splat.getSpectrum(spt='M7',lucky=True)[0]
sp1.normalize()
sp2.normalize()

# subtract
sp3 = sp1-sp2

# plot the individual spectra and their difference in two panels
plt.subplot(211)
plt.plot(sp1.wave,sp1.flux,'b-')
plt.plot(sp2.wave,sp2.flux,'g-')
plt.ylim([0,1.2])
plt.xlim([0.8,2.4])
plt.ylabel('Normalized Flux Density')
plt.legend([sp1.name,sp2.name])

plt.subplot(212)
plt.plot(sp3.wave,sp3.flux,'k-')
plt.legend(['Difference'])
plt.plot([0.8,2.4],[0,0],'k--')
plt.fill_between(sp3.wave,sp3.noise,-1.*sp3.noise,color='k',alpha=0.3)
plt.ylim([-0.5,0.5])
plt.xlim([0.8,2.4])
plt.xlabel('Wavelength (micron)')
plt.ylabel('Difference')


In [ ]:
# fit part of a spectrum to a line and divide this out
fit_range = [0.8,1.15]

# read in an L dwarf spectrum and trim 
sp = splat.getSpectrum(spt='L4',snr=40,lucky=True)[0]
sp.trim(fit_range)

# fit to a line using np.polyfit
fit = np.polyfit(sp.wave.value,sp.flux.value,1)

# generate a spectrum that is this linear function
sp_continuum = splat.Spectrum(wave=sp.wave,flux=np.polyval(fit,sp.wave.value)*sp.flux.unit)

# divide out this continuum
sp_normalized = sp/sp_continuum

# plot the results
plt.subplot(211)
plt.plot(sp.wave,sp.flux,'k-')
plt.plot(sp_continuum.wave,sp_continuum.flux,'g-')
plt.ylim([0,np.nanquantile(sp.flux.value,0.98)*1.5])
plt.xlim(fit_range)
plt.ylabel('Normalized Flux Density')
plt.legend([sp.name,'Continuum'])

plt.subplot(212)
plt.plot(sp_normalized.wave,sp_normalized.flux,'k-')
plt.legend(['Continuum-Corrected'])
plt.plot(fit_range,[1,1],'k--')
plt.ylim([0.5,1.5])
plt.xlim(fit_range)
plt.xlabel('Wavelength (micron)')
plt.ylabel('Ratio')


In [ ]:
# Here's a real science case:
# we're going to compare the spectrum of a known unresolved binary 2MASS J0518-2828 to a spectral
# template constructed from two differently-classified sources.
# Read in an L5 dwarf and a T5 dwarf, each with a minimum signal-to-noise and not known to be binary
# Scale these to their J-band absolute magnitudes and add them together to get a spectral binary
# We're going to use the empirical package in splat to get these absolute magnitudes

# relevant information
binary_name = 'J0518-2828'
pspt = 'L5'
sspt = 'T5'
phot_filter = '2MASS J'

# read in spectrum of known spectral binary
sp = splat.getSpectrum(shortname=binary_name)[0]

# read in template spectra
sp1 = splat.getSpectrum(spt=pspt,snr=20,binary=False,lucky=True)[0]
sp2 = splat.getSpectrum(spt=sspt,snr=20,binary=False,lucky=True)[0]

# get the right magnitudes from an empirical relation of Filippazzo et al. (2015)
# this returns the value and uncertainty
mag1 = spem.typeToMag(pspt,phot_filter,set='filippazzo')
mag2 = spem.typeToMag(sspt,phot_filter,set='filippazzo')
print(mag1,mag2)

# scale the spectra
sp1.fluxCalibrate(phot_filter,mag1[0],absolute=True)
sp2.fluxCalibrate(phot_filter,mag2[0],absolute=True)

# add them
sp3 = sp1+sp2

# normalize original spectrum to match at peak
sp.scale(sp3.fluxMax(wrange=[1.25,1.3])/sp.fluxMax(wrange=[1.25,1.3]))

# compute the difference
spdiff = sp-sp3

# visualize the results
plt.figure(figsize=[6,10])
plt.subplot(211)
plt.plot(sp.wave,sp.flux,'k-')
plt.plot(sp1.wave,sp1.flux,'m-')
plt.plot(sp2.wave,sp2.flux,'b-')
plt.plot(sp3.wave,sp3.flux,'g-')
plt.legend([binary_name,pspt,sspt,'Combined'])
plt.ylim([0,np.nanquantile(sp3.flux.value,0.98)*1.5])
plt.xlim([0.8,2.4])
plt.ylabel('Normalized Flux Density')

plt.subplot(212)
plt.plot(spdiff.wave,spdiff.flux,'k-')
plt.fill_between(spdiff.wave,spdiff.noise,-1.*spdiff.noise,color='k',alpha=0.3)
plt.legend(['Difference','Uncertainty'])
plt.plot([0.8,2.4],[0,0],'k--')
plt.ylim([x*np.nanquantile(sp3.flux.value,0.98) for x in [-0.5,0.5]])
plt.xlim([0.8,2.4])
plt.xlabel('Wavelength (micron)')
plt.ylabel('Difference')


# Index measurement

SPLAT has built-in functions to do index measurement, including literature-defined index sets and empirical relations to turn these into classifications

In [ ]:
# do a basic index measurement
# read in a random T5
sp = splat.getSpectrum(spt='T5',lucky=True)[0]

# measure the ratio of two regions - first range is numerator second range is denominator
ind = splat.measureIndex(sp,[[1.1,1.2],[1.22,1.32]],method='ratio',sample='integrate')
print(ind)

In [ ]:
# you can visualize the placement of these indices by setting plot=True
# NOTE: this is throwing some errors
ind = splat.measureIndex(sp,[[1.1,1.2],[1.22,1.32]],method='ratio',sample='integrate',plot=True)


In [ ]:
# measure an index set that is pre-defined in the literature
# this returns a dictionary of values
splat.measureIndexSet(sp,ref='burgasser',verbose=True)

In [ ]:
# there is a handy information function to find out what index sets are currently included
spem.info_indices()

In [ ]:
# you can use these indices to classify an object
splat.classifyByIndex(sp,ref='burgasser',verbose=True)

In [ ]:
# indices are also used for gravity classification of young sources
sp = splat.getSpectrum(spt='L2',young=True,lucky=True)[0]
splat.classifyGravity(sp,verbose=True)

# Comparing spectra and spectral classification

We often want to compare spectra against each other, either to classify or to fit to models. The main function to do this is splat.compareSpectra, which returns the comparison statistic and optimal scale factor, and has many options for modifying and visualizing the comparison.

In [ ]:
# check out the options of compareSpectra
splat.compareSpectra?

In [ ]:
# read in M7 and M8 spectra and compare them
sp1 = splat.getSpectrum(spt='M7',lucky=True)[0]
sp2 = splat.getSpectrum(spt='M8',lucky=True)[0]

splat.compareSpectra(sp1,sp2,plot=True)


In [ ]:
# limit copmarison to a specific range
splat.compareSpectra(sp1,sp2,fit_ranges=[0.8,1.0],plot=True)


In [ ]:
# to compare to spectral standards, you can use the built-in list of these standards
splat.initializeStandards()  # first read in the standards
stdM8 = splat.STDS_DWARF_SPEX['M8.0']  # there are different standard for different instruments
splat.compareSpectra(sp2,stdM8,plot=True)


In [ ]:
# a more efficient way to accomplish this is to use splat.classifyByStandard which will find the best match
splat.classifyByStandard(sp2,plot=True)

In [ ]:
# you can compare to alternate standards as well
# this command compares to a suite of subdwarf standards
splat.classifyByStandard(sp2,plot=True,sd=True)

In [ ]:
# this command compares to a suite of low gravity standards
splat.classifyByStandard(sp2,plot=True,vlg=True)